In [177]:
import numpy as np
from Assignments.Third.Model import MyFFLM, cross_entropy
import re

In [178]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

with open("lotrFotr.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()

stripped_txt_lotr = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').replace('"', '').lower()
stripped_txt_lotr = re.sub(r'[^\w\s]', '', stripped_txt_lotr).split()

with open("mobydick.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()

stripped_txt_mb = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').lower().split()

In [179]:
vocab = list(set(stripped_txt_lotr))
vocab.sort()
vocab_dict = {}
for i, word in enumerate(vocab):
    identity_vector = np.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[word] = identity_vector
print(len(vocab))

9794


In [180]:
# Need to split up bc of ram issues, batches is the answer I think
memory_depth = 3
dataX = np.zeros((len(stripped_txt_lotr[:1000]) - memory_depth, memory_depth, len(vocab_dict)))
dataY = np.zeros(len(stripped_txt_lotr[:1000]) - memory_depth, dtype='int32')
for i in range(len(stripped_txt_lotr[:1000]) - memory_depth):
    x = np.zeros((memory_depth, len(vocab_dict)))
    for j in range(memory_depth):
        x[j] = vocab_dict[stripped_txt_lotr[:1000][i+j]]
    dataX[i] = x
    dataY[i] = np.argmax(vocab_dict[stripped_txt_lotr[:1000][i+memory_depth]])

In [181]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[np.argmax(vocab_dict[key])] = key
for x in dataX[0]:
    print(reversed_dict[np.argmax(x)])
    print(x)
print(reversed_dict[dataY[0]])
print(dataY[0])

three
[0. 0. 0. ... 0. 0. 0.]
rings
[0. 0. 0. ... 0. 0. 0.]
for
[0. 0. 0. ... 0. 0. 0.]
the
8463


In [ ]:
v_len = len(vocab)
data_set_length = len(stripped_txt_lotr) - memory_depth
batch_size = 1000
model = MyFFLM(v_len, 64, learning_rate=0.001, memory_depth=memory_depth)
# loss = 0
for epoch in range(60):
    for batch in range(data_set_length // batch_size):
        dataX = np.zeros((batch_size - memory_depth, memory_depth, len(vocab_dict)))
        dataY = np.zeros(batch_size - memory_depth, dtype='int32')
        for i in range(batch_size - memory_depth):
            x = np.zeros((memory_depth, len(vocab_dict)))
            for j in range(memory_depth):
                x[j] = vocab_dict[stripped_txt_lotr[batch:batch+batch_size][i+j]]
            dataX[i] = x
            dataY[i] = np.argmax(vocab_dict[stripped_txt_lotr[batch:batch+batch_size][i+memory_depth]])
        for k, (x, y) in enumerate(zip(dataX, dataY)):
            # print(k, x, y)
            y_t = np.zeros(v_len)
            y_t[y] = 1
            # print(y_t, np.argmax(y_t))
            y_pred = model.forward(x)
            # loss += 0.01 * (cross_entropy(y_t, y_pred) - loss)
            model.backprop(y_t, y_pred)
            if k % 998 == 1:
                print("epoch:", epoch, "batch:", f"{batch}/{data_set_length//batch_size}")
                l = cross_entropy(y_t, y_pred)
                print("loss:", l, l[0, np.argmax(l)])
                print("Predicted vs actual:", reversed_dict[np.argmax(y_pred)], reversed_dict[y])
                print("Predicted value:", y_pred[0, np.argmax(y_pred)])


epoch: 0 batch: 0/187
loss: [[nan nan nan ... nan nan nan]] nan
Predicted vs actual: irresistibly elvenkings
Predicted value: 1.0
epoch: 0 batch: 1/187
loss: [[-1.38643102e-04 -1.53277723e-04 -1.11616147e-04 ... -1.35855697e-04
  -7.46994364e-05 -6.24829444e-05]] 9.483912174747084
Predicted vs actual: hills under
Predicted value: 0.00016113631903191973
epoch: 0 batch: 2/187
loss: [[-1.38644788e-04 -1.53277397e-04 -1.11616033e-04 ... -1.35855477e-04
  -7.46994111e-05 -6.24829336e-05]] 9.35862352015115
Predicted vs actual: hills the
Predicted value: 0.00016113593688307475
